In [1]:
import cx_Oracle
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
def connect_to_DB (username,password,hostname):
    """ Returns a connection to Oracle database"""
    try:
        connection = cx_Oracle.connect(username, password, hostname, encoding="UTF-8")
        print  ("Successffuly connected to the database")
    except:
        raise Exception('Connection failed! Please verifiy your login parameters')

    return connection

In [ ]:
# Connect to BCGW
hostname = 'bcgw.bcgov/idwprod1.bcgov'
username = input("Enter your BCGW username:")
password = input("Enter your BCGW password:")

connection = connect_to_DB (username,password,hostname)

In [4]:
# Get the Geometry column name.
# Added this because the geometry column name of BCGW tables can be SHAPE or GEOMETRY.

geom_query =  """
                SELECT column_name GEOM_NAME
                FROM  ALL_SDO_GEOM_METADATA
                WHERE owner = 'WHSE_FOREST_VEGETATION'
                   AND table_name = 'RSLT_OPENING_SVW'
              """
df_col = df = pd.read_sql(geom_query, con=connection)

geoCol_name = df_col ['GEOM_NAME'].iloc[0]
geoCol_name

'GEOMETRY'

In [5]:
# SQL query to execute
query ="""
           SELECT f.OPENING_ID, f.OPENING_LOCATION_NAME,
                  TO_CHAR(SDO_UTIL.TO_WKTGEOMETRY(f.{geom})) GEO, 
                  f.{geom}.sdo_srid SRID
                  
           FROM WHSE_FOREST_VEGETATION.RSLT_OPENING_SVW f
           
           WHERE f.OPENING_ID in (24962,24970,24963,24971) 
    """ .format (geom = geoCol_name)

In [6]:
# Read query results into df
df = pd.read_sql(query, con=connection)

# Format the Geometry column
df['geometry'] = df['GEO'].apply(wkt.loads)

In [7]:
# Create a gdf and set a 
gdf = gpd.GeoDataFrame(df, geometry = df['geometry'])
crs = str(df['SRID'].iloc[0])[-4:]
gdf.crs = "EPSG:" + crs

del df['SRID']
del df['GEO']

gdf

,OPENING_ID,OPENING_LOCATION_NAME,geometry
0,24962,BUSH CHAPERON,"POLYGON ((1586440.054 780765.616, 1586414.831 ..."
1,24963,BUSH CHAPERON,"POLYGON ((1586897.200 780087.449, 1586867.007 ..."
2,24970,BUSH CHAPERON,"POLYGON ((1585564.343 779928.522, 1585563.200 ..."
3,24971,BUSH CHAPERON,"POLYGON ((1586373.591 780396.240, 1586377.056 ..."


In [10]:
# Export to shp (or other format)
# For supported formats. run: fiona.supported_drivers

file_format = "ESRI Shapefile"
file_name = r'F:\tko_root\GIS_WORKSPACE\MLABIADH\GIS-REQUESTS\tempo\test_poly.shp'
gdf.to_file(file_name, driver=file_format)